# 436V Crime in Vancouver Data Preprocessing

## Imports 

In [1]:
import os

%matplotlib inline
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import utm

# suppress warning
import warnings
warnings.filterwarnings("ignore")

## preprocessing

### all years

In [2]:
# get the data from 2015 - 2020
df_2017_2020 = pd.read_csv("crimedata_all_years.csv")

# get rid of rows with NAN
df_2017_2020 = df_2017_2020.sort_values(["YEAR"], ascending=True)
df_2017_2020 = df_2017_2020.query('YEAR >= 2017 and YEAR <= 2020')

### 2021 specific

In [3]:
# get the data in 2021
df_2021 = pd.read_csv("crimedata_2021.csv")

### combined

In [4]:
df_combined = pd.concat([df_2017_2020, df_2021])
df_combined.index.name = "CASE_ID"

### Preprocessing

#### 1. get rid of rows with NEIGHBOURHOOD is NAN, and X and Y is 0.0

In [5]:
df_combined = df_combined.query('NEIGHBOURHOOD.isnull() == False')
df_combined.drop(df_combined[df_combined.X == 0.0].index | df_combined[df_combined.Y == 0.0].index, inplace=True)
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 162300 entries, 540730 to 32035
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   TYPE           162300 non-null  object 
 1   YEAR           162300 non-null  int64  
 2   MONTH          162300 non-null  int64  
 3   DAY            162300 non-null  int64  
 4   HOUR           162300 non-null  int64  
 5   MINUTE         162300 non-null  int64  
 6   HUNDRED_BLOCK  162300 non-null  object 
 7   NEIGHBOURHOOD  162300 non-null  object 
 8   X              162300 non-null  float64
 9   Y              162300 non-null  float64
dtypes: float64(2), int64(5), object(3)
memory usage: 13.6+ MB


#### 2. get rid of rows whose neighborhood is Musqueam and Stanley Park

In [6]:
nei1 = "Musqueam"
nei2 = "Stanley Park"
df_combined = df_combined.query('NEIGHBOURHOOD != @nei1 and NEIGHBOURHOOD != @nei2')
df_combined.drop(df_combined[df_combined.NEIGHBOURHOOD == nei1].index | df_combined[df_combined.NEIGHBOURHOOD == nei2].index, inplace=True)
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 161532 entries, 540730 to 32035
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   TYPE           161532 non-null  object 
 1   YEAR           161532 non-null  int64  
 2   MONTH          161532 non-null  int64  
 3   DAY            161532 non-null  int64  
 4   HOUR           161532 non-null  int64  
 5   MINUTE         161532 non-null  int64  
 6   HUNDRED_BLOCK  161532 non-null  object 
 7   NEIGHBOURHOOD  161532 non-null  object 
 8   X              161532 non-null  float64
 9   Y              161532 non-null  float64
dtypes: float64(2), int64(5), object(3)
memory usage: 13.6+ MB


In [7]:
df_combined['NEIGHBOURHOOD'].nunique()

22

#### 3. convert X, Y to lat and long

In [8]:
def rule(row):
    lat, long = utm.to_latlon(row["X"], row["Y"], 10, 'U')
    return pd.Series({"lat": lat, "long": long})

In [9]:
df_lat_long = df_combined.apply(rule, axis=1)

In [10]:
df_combined = pd.concat([df_combined, df_lat_long], axis=1)

In [11]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 161532 entries, 540730 to 32035
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   TYPE           161532 non-null  object 
 1   YEAR           161532 non-null  int64  
 2   MONTH          161532 non-null  int64  
 3   DAY            161532 non-null  int64  
 4   HOUR           161532 non-null  int64  
 5   MINUTE         161532 non-null  int64  
 6   HUNDRED_BLOCK  161532 non-null  object 
 7   NEIGHBOURHOOD  161532 non-null  object 
 8   X              161532 non-null  float64
 9   Y              161532 non-null  float64
 10  lat            161532 non-null  float64
 11  long           161532 non-null  float64
dtypes: float64(4), int64(5), object(3)
memory usage: 16.0+ MB


#### 4. drop X, Y, HUNDRED_BLOCK, MINUTE

In [12]:
col = df_combined.columns
col

Index(['TYPE', 'YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', 'HUNDRED_BLOCK',
       'NEIGHBOURHOOD', 'X', 'Y', 'lat', 'long'],
      dtype='object')

In [13]:
df_combined = df_combined.drop(columns=['X', 'Y', 'HUNDRED_BLOCK', 'MINUTE'])
df_combined.columns

Index(['TYPE', 'YEAR', 'MONTH', 'DAY', 'HOUR', 'NEIGHBOURHOOD', 'lat', 'long'], dtype='object')

#### 5. add day and night column

In [14]:
# create a list of our conditions
conditions = [
    (df_combined['HOUR'] >= 6) & (df_combined['HOUR'] <= 18),
    ((df_combined['HOUR'] > 18) & (df_combined['HOUR'] <= 24)) 
    | ((df_combined['HOUR'] < 6) & (df_combined['HOUR'] >= 0)),
]

# create a list of the values we want to assign for each condition
values = ['day', 'night']

# create a new column and use np.select to assign values to it using our lists as arguments
df_combined['TIME'] = np.select(conditions, values)
df_combined.head()

,TYPE,YEAR,MONTH,DAY,HOUR,NEIGHBOURHOOD,lat,long,TIME
CASE_ID,,,,,,,,,
540730,Theft from Vehicle,2017,10,10,5,Grandview-Woodland,49.271372,-123.069626,night
168234,Mischief,2017,10,23,9,Central Business District,49.281187,-123.116062,day
95346,Break and Enter Residential/Other,2017,9,29,11,Sunset,49.231939,-123.077154,day
13124,Break and Enter Commercial,2017,11,3,4,Mount Pleasant,49.262964,-123.104498,night
22890,Break and Enter Commercial,2017,8,19,19,Fairview,49.257298,-123.124314,night


#### 6. aggregated attribute NumPerNeighbourhood of each year

In [15]:
nei_df_each_yr = pd.DataFrame(df_combined.groupby(['NEIGHBOURHOOD', 'YEAR']).size())
nei_df_each_yr.head(14)

0
NEIGHBOURHOOD             YEAR       
Arbutus Ridge             2017    377
                          2018    311
                          2019    295
                          2020    327
                          2021    273
Central Business District 2017   9950
                          2018  10839
                          2019  12362
                          2020   7690
                          2021   7836
Dunbar-Southlands         2017    457
                          2018    474
                          2019    414
                          2020    429

In [16]:
nei_df_each_yr_unstack = nei_df_each_yr.unstack()
nei_df_each_yr_unstack_trans = nei_df_each_yr_unstack.transpose().reset_index(level=0,drop=True).transpose()
nei_df_each_yr_unstack_trans = nei_df_each_yr_unstack_trans.rename(columns=
                                                                   {
                                                                       2017: "NumPerNeighbourhood_2017",
                                                                       2018: "NumPerNeighbourhood_2018",
                                                                       2019: "NumPerNeighbourhood_2019",
                                                                       2020: "NumPerNeighbourhood_2020",
                                                                       2021: "NumPerNeighbourhood_2021",
                                                                   })
nei_df_each_yr_unstack_trans.head(5)

YEAR,NumPerNeighbourhood_2017,NumPerNeighbourhood_2018,NumPerNeighbourhood_2019,NumPerNeighbourhood_2020,NumPerNeighbourhood_2021
NEIGHBOURHOOD,,,,,
Arbutus Ridge,377,311,295,327,273
Central Business District,9950,10839,12362,7690,7836
Dunbar-Southlands,457,474,414,429,306
Fairview,1735,1771,2117,1719,1487
Grandview-Woodland,1888,1719,1803,1405,1378


In [17]:
merged = pd.merge(df_combined, nei_df_each_yr_unstack_trans, on='NEIGHBOURHOOD')
merged.index.name="CASE_ID"

#### 7. aggregated attribute NumPerNeighbourhood for ALL years

In [18]:
nei_df_all_yr = df_combined.groupby(by='NEIGHBOURHOOD').agg('count')['TYPE']
nei_df_all_yr = pd.DataFrame(nei_df_all_yr)
nei_df_all_yr = nei_df_all_yr.rename(columns={"TYPE": "NumPerNeighbourhood_all_yrs"})
nei_df_all_yr.head(5)

,NumPerNeighbourhood_all_yrs
NEIGHBOURHOOD,
Arbutus Ridge,1583
Central Business District,48677
Dunbar-Southlands,2080
Fairview,8829
Grandview-Woodland,8193


In [19]:
merged2 = pd.merge(merged, nei_df_all_yr, on='NEIGHBOURHOOD')
merged2.index.name="CASE_ID"
merged2.head()

,TYPE,YEAR,MONTH,DAY,HOUR,NEIGHBOURHOOD,lat,long,TIME,NumPerNeighbourhood_2017,NumPerNeighbourhood_2018,NumPerNeighbourhood_2019,NumPerNeighbourhood_2020,NumPerNeighbourhood_2021,NumPerNeighbourhood_all_yrs
CASE_ID,,,,,,,,,,,,,,,
0,Theft from Vehicle,2017,10,10,5,Grandview-Woodland,49.271372,-123.069626,night,1888,1719,1803,1405,1378,8193
1,Theft from Vehicle,2017,2,22,19,Grandview-Woodland,49.265787,-123.069723,night,1888,1719,1803,1405,1378,8193
2,Theft of Bicycle,2017,4,21,13,Grandview-Woodland,49.281213,-123.058524,day,1888,1719,1803,1405,1378,8193
3,Theft from Vehicle,2017,1,16,12,Grandview-Woodland,49.271372,-123.069626,day,1888,1719,1803,1405,1378,8193
4,Theft from Vehicle,2017,9,17,18,Grandview-Woodland,49.266874,-123.069700,day,1888,1719,1803,1405,1378,8193


#### 8. check the sum of each year is equal to all_yrs

In [20]:
check_list = (merged2["NumPerNeighbourhood_all_yrs"] == (
    merged2["NumPerNeighbourhood_2017"] +
    merged2["NumPerNeighbourhood_2018"] + 
    merged2["NumPerNeighbourhood_2019"] + 
    merged2["NumPerNeighbourhood_2020"] + 
    merged2["NumPerNeighbourhood_2021"]
    ))
print(len(check_list))
for l in check_list:
    if l != True:
        print("damn")
print("yay")

161532
yay


#### 9. check merged2

In [21]:
merged2.head()

,TYPE,YEAR,MONTH,DAY,HOUR,NEIGHBOURHOOD,lat,long,TIME,NumPerNeighbourhood_2017,NumPerNeighbourhood_2018,NumPerNeighbourhood_2019,NumPerNeighbourhood_2020,NumPerNeighbourhood_2021,NumPerNeighbourhood_all_yrs
CASE_ID,,,,,,,,,,,,,,,
0,Theft from Vehicle,2017,10,10,5,Grandview-Woodland,49.271372,-123.069626,night,1888,1719,1803,1405,1378,8193
1,Theft from Vehicle,2017,2,22,19,Grandview-Woodland,49.265787,-123.069723,night,1888,1719,1803,1405,1378,8193
2,Theft of Bicycle,2017,4,21,13,Grandview-Woodland,49.281213,-123.058524,day,1888,1719,1803,1405,1378,8193
3,Theft from Vehicle,2017,1,16,12,Grandview-Woodland,49.271372,-123.069626,day,1888,1719,1803,1405,1378,8193
4,Theft from Vehicle,2017,9,17,18,Grandview-Woodland,49.266874,-123.069700,day,1888,1719,1803,1405,1378,8193


In [22]:
merged2 = merged2.sort_values(by=['YEAR', 'CASE_ID'])
merged2.head(20)

,TYPE,YEAR,MONTH,DAY,HOUR,NEIGHBOURHOOD,lat,long,TIME,NumPerNeighbourhood_2017,NumPerNeighbourhood_2018,NumPerNeighbourhood_2019,NumPerNeighbourhood_2020,NumPerNeighbourhood_2021,NumPerNeighbourhood_all_yrs
CASE_ID,,,,,,,,,,,,,,,
0,Theft from Vehicle,2017,10,10,5,Grandview-Woodland,49.271372,-123.069626,night,1888,1719,1803,1405,1378,8193
1,Theft from Vehicle,2017,2,22,19,Grandview-Woodland,49.265787,-123.069723,night,1888,1719,1803,1405,1378,8193
2,Theft of Bicycle,2017,4,21,13,Grandview-Woodland,49.281213,-123.058524,day,1888,1719,1803,1405,1378,8193
3,Theft from Vehicle,2017,1,16,12,Grandview-Woodland,49.271372,-123.069626,day,1888,1719,1803,1405,1378,8193
4,Theft from Vehicle,2017,9,17,18,Grandview-Woodland,49.266874,-123.069700,day,1888,1719,1803,1405,1378,8193
5,Theft from Vehicle,2017,11,11,0,Grandview-Woodland,49.273056,-123.074415,night,1888,1719,1803,1405,1378,8193
6,Theft from Vehicle,2017,12,7,14,Grandview-Woodland,49.273056,-123.074415,day,1888,1719,1803,1405,1378,8193
7,Theft from Vehicle,2017,10,3,22,Grandview-Woodland,49.266437,-123.069711,night,1888,1719,1803,1405,1378,8193
8,Theft of Vehicle,2017,10,28,22,Grandview-Woodland,49.283474,-123.072512,night,1888,1719,1803,1405,1378,8193


In [24]:
merged2.describe(include='all')

,TYPE,YEAR,MONTH,DAY,HOUR,NEIGHBOURHOOD,lat,long,TIME,NumPerNeighbourhood_2017,NumPerNeighbourhood_2018,NumPerNeighbourhood_2019,NumPerNeighbourhood_2020,NumPerNeighbourhood_2021,NumPerNeighbourhood_all_yrs
count,161532,161532.000000,161532.00000,161532.000000,161532.000000,161532,161532.000000,161532.000000,161532,161532.000000,161532.000000,161532.000000,161532.000000,161532.000000,161532.000000
unique,9,NaN,NaN,NaN,NaN,22,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN
top,Theft from Vehicle,NaN,NaN,NaN,NaN,Central Business District,NaN,NaN,day,NaN,NaN,NaN,NaN,NaN,NaN
freq,62094,NaN,NaN,NaN,NaN,48677,NaN,NaN,90263,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,2018.897773,6.57445,15.251888,13.409634,NaN,49.265194,-123.106336,NaN,4099.245165,4395.653592,4933.435010,3279.340261,3328.516969,20036.190996
std,NaN,1.377839,3.45053,8.698208,6.784516,NaN,0.021374,0.035287,NaN,3890.328771,4280.456865,4929.000162,2940.549561,3006.741759,19040.927436
min,NaN,2017.000000,1.00000,1.000000,0.000000,NaN,49.201201,-123.224021,NaN,347.000000,292.000000,295.000000,316.000000,273.000000,1583.000000
25%,NaN,2018.000000,4.00000,8.000000,9.000000,NaN,49.256401,-123.125703,NaN,1355.000000,1379.000000,1553.000000,1274.000000,1290.000000,6893.000000
50%,NaN,2019.000000,7.00000,15.000000,15.000000,NaN,49.272322,-123.112383,NaN,2012.000000,1985.000000,2163.000000,1793.000000,1892.000000,9428.000000
75%,NaN,2020.000000,10.00000,22.000000,19.000000,NaN,49.281781,-123.088811,NaN,9950.000000,10839.000000,12362.000000,7690.000000,7836.000000,48677.000000


### export to csv

In [26]:
merged2.to_csv('crime_2017_2021.csv')